<a href="https://colab.research.google.com/github/1900690/yolo-traking/blob/main/yolo_traking_apple_grape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#YOLOを使った追跡&数を数える


In [14]:
#@title 分析する動画をアップロードまたはサンプル動画(リンゴ)で分析

#事前準備
!pip -q install ultralytics

from google.colab import files
import os
リンゴのサンプル動画を使用する = True # @param {type:"boolean"}

if リンゴのサンプル動画を使用する == True:
  if os.path.exists('/content/ringo.mp4'):
     os.remove('/content/ringo.mp4')
  !wget -q  https://github.com/1900690/yolo-traking/releases/download/sample/30fps_ringo.mp4

else:
  uploaded_file = files.upload()
  uploaded_file_name = next(iter(uploaded_file))

<p align="center">
  <img src="https://github.com/1900690/yolo-traking/blob/main/images/ringo_before.png?raw=1" width=50%>
  <br>
  リンゴのサンプル動画
</p>

#それぞれの対象を追跡する（数については正確性が乏しい）

In [16]:
#@markdown 分析して動画をダウンロード
#因数は以下を参照https://docs.ultralytics.com/modes/predict/#inference-arguments
from ultralytics import YOLO
import shutil
import os
from google.colab import files

if os.path.exists('/content/runs'):
    shutil.rmtree('/content/runs')

# Load a model
model = YOLO("yolo11n.pt")

if リンゴのサンプル動画を使用する == True:
  results = model.track(source="/content/30fps_ringo.mp4",stream=False,save=True,classes=47,save_txt=True,show_conf=False)
  files.download("/content/runs/detect/track/30fps_ringo.avi")
else:
  results = model.track(source="/content/"+uploaded_file_name,stream=False,save=True,save_txt=True,show_conf=False)
  files.download("/content/runs/detect/track/"+uploaded_file_name[:-4]+".avi")



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/557) /content/30fps_ringo.mp4: 384x640 23 apples, 96.6ms
video 1/1 (frame 2/557) /content/30fps_ringo.mp4: 384x640 23 apples, 9.7ms
video 1/1 (frame 3/557) /content/30fps_ringo.mp4: 384x640 23 apples, 12.7ms
video 1/1 (frame 4/557) /content/30fps_ringo.mp4: 384x640 23 apples, 11.1ms
video 1/1 (frame 5/557) /content/30fps_ringo.mp4: 384x640 23 apples, 9.7ms
video 1/1 (frame 6/557) /content/30fps_ringo.mp4: 384x640 23 apples, 8.9ms
vid

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<p align="center">
  <img src="https://github.com/1900690/yolo-traking/blob/main/images/ringo_after.png?raw=1" width=50%>
  <br>
  果実数を検出後のリンゴのサンプル動画
</p>

#特定の範囲を通り過ぎたものの数をカウントする

<p align="center">
  <img src="https://github.com/1900690/yolo-traking/blob/main/images/muscut3D_before.png?raw=1" width=30%>
  <br>
  ブドウのサンプル動画
</p>

In [ ]:
#@title 分析する動画をアップロードまたはサンプル動画(ブドウ)で分析

#事前準備
!pip -q install ultralytics
!wget -q  https://github.com/1900690/yolo-traking/releases/download/sample/grape.pt

from google.colab import files
import os
サンプル動画を使用する = True # @param {type:"boolean"}

if サンプル動画を使用する == True:
  if os.path.exists('/content/muscat.mp4'):
     os.remove('/content/muscut.mp4')
  !wget -q  https://github.com/1900690/yolo-traking/releases/download/sample/muscat.mp4
else:
  uploaded_file = files.upload()
  uploaded_file_name_count = next(iter(uploaded_file))

In [4]:
import cv2

from ultralytics import solutions

cap = cv2.VideoCapture("/content/muscat.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Define region points
#ある線上を通り過ぎた数をカウントする場合
region_points = [(433, 0), (433, 866)]
#ある範囲内に出入りした数をカウントする場合
#region_points = [(0, 720), (1280, 720), (1280, 0), (0, 0)]  # For rectangle region counting
# region_points = [(20, 400), (1080, 404), (1080, 360), (20, 360), (20, 400)]  # For polygon region counting

# Video writer
video_writer = cv2.VideoWriter("/content/muscat.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init Object Counter
counter = solutions.ObjectCounter(
    #show=True,  # Display the output
    region=region_points,  # Pass region points
    model="/content/grape.pt",  # model="yolo11n-obb.pt" for object counting using YOLO11 OBB model.
    #classes=[47],  # If you want to count specific classes i.e person and car with COCO pretrained model.
    show_in=False,  # Display in counts
    # show_out=True,  # Display out counts
    # line_width=2,  # Adjust the line width for bounding boxes and text display
)

# Process video
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    im0 = counter.count(im0)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

Ultralytics Solutions: ✅ {'region': [(433, 0), (433, 866)], 'show_in': False, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'model': '/content/grape.pt'}
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()


0: 640x640 23 rests, 11.8ms
Speed: 3.7ms preprocess, 11.8ms inference, 578.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 23 rests, 9.6ms
Speed: 6.0ms preprocess, 9.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 23 rests, 9.2ms
Speed: 4.4ms preprocess, 9.2ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 24 rests, 8.3ms
Speed: 4.6ms preprocess, 8.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 24 rests, 8.4ms
Speed: 3.2ms preprocess, 8.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 24 rests, 8.3ms
Speed: 5.0ms preprocess, 8.3